In [11]:
import pandas as pd
import numpy as np

# set the seed
np.random.seed(99)

In [12]:
from fb_prophet import *

In [13]:
# File path to the CSV data
file_path = '../data/Day Ahead Auction Prices.csv'

# Load and preprocess the data
day_ahead = load_day_ahead_prices(file_path)

# Prepare the DataFrame for Prophet
df = day_ahead.reset_index().rename(columns={'datetime': 'ds', 'price': 'y'})
df

,ds,y
0,2016-01-01 00:00:00,23.86
1,2016-01-01 01:00:00,22.39
2,2016-01-01 02:00:00,20.59
3,2016-01-01 03:00:00,16.81
4,2016-01-01 04:00:00,17.41
...,...,...
73771,2024-05-31 19:00:00,107.94
73772,2024-05-31 20:00:00,115.00
73773,2024-05-31 21:00:00,107.91
73774,2024-05-31 22:00:00,101.74


In [14]:
# Get the last date in the dataset
last_date = df['ds'].max()

# Define train-test split
train_end_date = last_date - pd.Timedelta(days=365)
train_df = df[df['ds'] <= train_end_date]
test_df = df[df['ds'] > train_end_date]

# Define the parameter grid
param_grid = {
    'growth': ['flat'],
    'yearly_seasonality': [True],
    'weekly_seasonality': [True],
    'daily_seasonality': [True],
    'seasonality_mode': ['additive', 'multiplicative'],
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.1, 1, 10],
    'holidays_prior_scale': [0.01, 0.1, 1, 10]
}

In [15]:
# Perform hyperparameter tuning
horizon = '365 days'
tuning_results = tune_hyperparameters(train_df, param_grid, horizon)
tuning_results

In [ ]:
# Fit the best model
best_params = tuning_results.loc[tuning_results['rmse'].idxmin()].to_dict()
m = Prophet(**{k: v for k, v in best_params.items() if k != 'rmse'})
m.add_country_holidays(country_name='DE')
m.fit(train_df)
print("best_params", best_params)

In [ ]:
# Forecast future values
future = m.make_future_dataframe(periods=365 * 24, freq='h')  # 1-year forecast
forecast = m.predict(future)

# Evaluate model on test set
y_test = test_df['y'].values
y_pred = forecast.loc[forecast['ds'].isin(test_df['ds']), 'yhat'].values

metrics = evaluate_model(y_test, y_pred)
print("Evaluation Metrics:", metrics)

In [ ]:
# Refit the model on the entire dataset
print("Refitting the model on the entire dataset for a 5-year forecast...")
m_full = Prophet(**{k: v for k, v in best_params.items() if k != 'rmse'}).
m_full.add_country_holidays(country_name='DE')
m_full.fit(df)

# Forecast 5 years into the future
future_full = m_full.make_future_dataframe(periods=5 * 365 * 24, freq='h')  # 5-year forecast
forecast_full = m_full.predict(future_full)

# Save the 5-year forecast to a CSV file
forecast_full.to_csv('../data/fb_prophet_5year_forecast.csv', index=False)
print("5-year forecast saved as fb_prophet_5year_forecast.csv")

# Plot the training data, 1-year forecast, and 5-year forecast
plot_training_and_forecast(train_df, forecast, forecast_full)

In [ ]:
mfull.plot_components(forecast_full)